In [ ]:
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler



/ictstr01/home/icb/eirini.giannakoulia/tools/apps/mamba/envs/mini/lib/python3.12/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/ictstr01/home/icb/eirini.giannakoulia/tools/apps/mamba/envs/mini/lib/python3.12/site-packages/anndata/utils.py:429: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/ictstr01/home/icb/eirini.giannakoulia/tools/apps/mamba/envs/mini/lib/python3.12/site-packages/anndata/utils.py:429: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  warnings.warn(msg, FutureWarning)
/ictstr01/home/icb/eirini.giannakoulia/tools/apps/mamba/envs/mini/lib/python3.12/site-packages/anndata/utils.py:429: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(ms

### xgboost	
hvg	{'alpha': 50, 'lambda': 100, 'max_depth': 3, 'learning_rate': 0.05, 'n_estimators': 1000, 'subsample': 0.8, 'colsample_bytree': 0.8, 'min_child_weight': 3, 'early_stopping_rounds': 10, 'n_jobs': 20}	0f20aea7b9
### lasso
svd	{'alpha': 0.1}	
hvg_svd	{'alpha': 0.1}	
hvg	{'alpha': 0.1}	
svd	{'alpha': 1.0}	
### ridge
svd	{'alpha': 0.1}	
hvg_svd	{'alpha': 1.0}	
### linear	
svd	{'parm': 0}	


In [2]:
adata_msi = sc.read('/lustre/groups/ml01/workspace/anastasia.litinetskaya/code/vitatrack/datasets/V11L12-038_A1.RNA_MOSCOT_paired_hvg.h5ad')
adata_rna = sc.read('/lustre/groups/ml01/workspace/anastasia.litinetskaya/code/vitatrack/datasets/V11L12-038_A1.RNA_MOSCOT_paired_hvg.h5ad')

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler

In [5]:
params = {}

In [ ]:
top_genes = params.get("top_genes", 5000)
n_components = params.get("n_components", 16)
split_name = "split"


Proccess MSI data with SVD

In [ ]:
if "highly_variable" not in adata_msi.var.columns:
    sc.pp.highly_variable_genes(adata_msi, flavor='seurat', n_top_genes=top_genes)
hvg_msi = adata_msi[:, adata_msi.var["highly_variable"]]

#split train and test, used for svd and rest
hvg_msi_train = hvg_msi[hvg_msi.obs[split_name] == "train"]
hvg_msi_test = hvg_msi[hvg_msi.obs[split_name] == "test"]

#-----SVD-----#
msi_svd_reducer = TruncatedSVD(n_components=n_components) 

svd_features_train = msi_svd_reducer.fit_transform(hvg_msi_train.X.toarray())
hvg_msi_train.obsm["svd_features"] = svd_features_train

svd_features_test = msi_svd_reducer.fit_transform(hvg_msi_test.X.toarray())
hvg_msi_test.obsm["svd_features"] = svd_features_test



/tmp/ipykernel_108321/212701971.py:13: ImplicitModificationWarning: Setting element `.obsm['svd_features']` of view, initializing view as actual.
  hvg_msi_train.obsm["svd_features"] = svd_features_train
/tmp/ipykernel_108321/212701971.py:16: ImplicitModificationWarning: Setting element `.obsm['svd_features']` of view, initializing view as actual.
  hvg_msi_test.obsm["svd_features"] = svd_features_test


Process seqRNA with SVD

In [ ]:
#Split into train test
rna_train = adata_rna[adata_rna.obs[split_name] == "train"]
rna_test = adata_rna[adata_rna.obs[split_name] == "test"]

#----------------------------------------------sc-seqRNA----------------------------------------------#
#-----SVD-----#
rna_svd_reducer = TruncatedSVD(n_components=n_components)

svd_features_train = rna_svd_reducer.fit_transform(rna_train.X.toarray())
rna_train.obsm["svd_features"] = svd_features_train

svd_features_test = rna_svd_reducer.fit_transform(rna_test.X.toarray())
rna_test.obsm["svd_features"] = svd_features_test

/tmp/ipykernel_108321/3468432298.py:10: ImplicitModificationWarning: Setting element `.obsm['svd_features']` of view, initializing view as actual.
  rna_train.obsm["svd_features"] = svd_features_train
/tmp/ipykernel_108321/3468432298.py:13: ImplicitModificationWarning: Setting element `.obsm['svd_features']` of view, initializing view as actual.
  rna_test.obsm["svd_features"] = svd_features_test


Fix first part of lasso to acommodate new changes:


In [9]:
featsel = "svd"

In [21]:
X_train = rna_train.obsm["svd_features"]
X_test = rna_test.obsm["svd_features"]
# Y_train, Y_test = hvg_msi_train.obsm["svd_features"], hvg_msi_test.obsm["svd_features"]
Y_train, Y_test = hvg_msi_train.X, hvg_msi_test.X


XGBOOST


In [12]:
import numpy as np
import pandas as pd
import scanpy as sc
import cupy as cp  # For GPU array conversion
from xgboost import XGBRegressor
from scipy.stats import pearsonr, spearmanr
from sklearn.metrics import root_mean_squared_error, r2_score  # Using MSE then taking sqrt
from scipy.sparse import issparse


In [13]:
def ensure_gpu(data):
    """
    Convert the data to a GPU array if it is not already.
    If the data is a pandas DataFrame or NumPy array, convert it to a CuPy array.
    """
    # If data already has the __cuda_array_interface__, assume it is a GPU array.
    if hasattr(data, '__cuda_array_interface__'):
        return data
    # If data is a DataFrame, convert its underlying NumPy array.
    if isinstance(data, pd.DataFrame):
        return cp.asarray(data.values)
    # If data is a NumPy array, convert it.
    if isinstance(data, np.ndarray):
        return cp.asarray(data)
    # Otherwise, attempt conversion.
    return cp.asarray(data)

def ensure_cpu(data):
    """
    Convert the data to a CPU (NumPy) array if it is on the GPU.
    """
    if hasattr(data, '__cuda_array_interface__'):
        return cp.asnumpy(data)
    return data


In [14]:
params = {'alpha': 50, 'lambda': 100, 'max_depth': 3, 'learning_rate': 0.05, 'n_estimators': 1000, 'subsample': 0.8, 'colsample_bytree': 0.8, 'min_child_weight': 3, 'early_stopping_rounds': 10, 'n_jobs': 20}

In [22]:
if issparse(X_train):
    X_train = X_train.toarray()
if issparse(X_test):
    X_test = X_test.toarray()
if issparse(Y_train):
    Y_train = Y_train.toarray()
if issparse(Y_test):
    Y_test = Y_test.toarray()

#Since the model is configured to run on GPU (device="cuda"),
#convert all input data to GPU arrays if they are not already.
#If you don't need that, you can just comment out the conversions
X_train = ensure_gpu(X_train)
X_test = ensure_gpu(X_test)
Y_train = ensure_gpu(Y_train)
Y_test = ensure_gpu(Y_test)

# XGBoost Hyperparameters
alpha = float(params.get("alpha", 10))         # L1 regularization (Lasso)
lambda_ = float(params.get("lambda", 50))       # L2 regularization (Ridge)
max_depth = int(params.get("max_depth", 5))
learning_rate = float(params.get("learning_rate", 0.1))
n_estimators = int(params.get("n_estimators", 500))
subsample = float(params.get("subsample", 0.9))
colsample_bytree = float(params.get("colsample_bytree", 0.7))
min_child_weight = int(params.get("min_child_weight", 2))
early_stopping_rounds = int(params.get("early_stopping_rounds", 20))
n_jobs = int(params.get("n_jobs", 15))


# Initialize XGBoost model on GPU
xgb_model = XGBRegressor(
    device="cuda",
    reg_alpha=alpha, 
    reg_lambda=lambda_,
    max_depth=max_depth,
    learning_rate=learning_rate,
    n_estimators=n_estimators,
    subsample=subsample,
    colsample_bytree=colsample_bytree,
    min_child_weight=min_child_weight,
    early_stopping_rounds=early_stopping_rounds,
    n_jobs=n_jobs
)


In [23]:
# Train the model
xgb_model.fit(
    X_train, 
    Y_train, 
    eval_set=[(X_test, Y_test)], 
    verbose=True
)

[0]	validation_0-rmse:0.34794
[1]	validation_0-rmse:0.34019
[2]	validation_0-rmse:0.33300
[3]	validation_0-rmse:0.32622
[4]	validation_0-rmse:0.31995
[5]	validation_0-rmse:0.31410
[6]	validation_0-rmse:0.30864
[7]	validation_0-rmse:0.30355
[8]	validation_0-rmse:0.29882
[9]	validation_0-rmse:0.29446
[10]	validation_0-rmse:0.29042
[11]	validation_0-rmse:0.28664
[12]	validation_0-rmse:0.28314
[13]	validation_0-rmse:0.27988
[14]	validation_0-rmse:0.27685
[15]	validation_0-rmse:0.27405
[16]	validation_0-rmse:0.27144
[17]	validation_0-rmse:0.26907
[18]	validation_0-rmse:0.26683
[19]	validation_0-rmse:0.26476
[20]	validation_0-rmse:0.26284
[21]	validation_0-rmse:0.26108
[22]	validation_0-rmse:0.25941
[23]	validation_0-rmse:0.25788
[24]	validation_0-rmse:0.25644
[25]	validation_0-rmse:0.25512
[26]	validation_0-rmse:0.25391
[27]	validation_0-rmse:0.25275
[28]	validation_0-rmse:0.25169
[29]	validation_0-rmse:0.25069
[30]	validation_0-rmse:0.24976
[31]	validation_0-rmse:0.24890
[32]	validation_0-

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device='cuda', early_stopping_rounds=10,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=3, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=20,
             num_parallel_tree=None, random_state=None, ...)

In [26]:

# Predict on the test data
Y_pred = xgb_model.predict(X_test)

# Convert predictions and test labels back to CPU (NumPy) for evaluation
Y_pred = ensure_cpu(Y_pred)
Y_test_cpu = ensure_cpu(Y_test)

# Pearson and Spearman correlation
pearson_corr = pearsonr(Y_pred.flatten(), Y_test_cpu.flatten())[0]
spearman_corr = spearmanr(Y_pred.flatten(), Y_test_cpu.flatten())[0]

# Compute Root Mean Squared Error and R2 score
rmse_test = root_mean_squared_error(Y_test_cpu, Y_pred)
r2_test = r2_score(Y_test_cpu, Y_pred)

#Save results to a DataFrame
metrics = pd.DataFrame({
    'rmse': [rmse_test],
    'r2': [r2_test],
    'pearson': [pearson_corr],
    'spearman': [spearman_corr]
})

#Add this for interpretability later, check outputs of each model's preds
predictions = pd.DataFrame({
    'y_true': Y_test_cpu.flatten(),
    'y_pred': Y_pred.flatten()
})


In [24]:
metrics_vitatrack_svd = metrics

In [28]:
metrics_vitatrack


,rmse,r2,pearson,spearman
0,0.194244,-0.166843,0.756211,0.401311


In [25]:
metrics_vitatrack_svd

,rmse,r2,pearson,spearman
0,0.556224,0.494908,0.980315,0.827191
